<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/well/wellcalcs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%capture
!pip install neqsim
import neqsim
from neqsim.thermo.thermoTools import *
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline

# Oil and gas Wells

#Litterature

https://www.youtube.com/watch?v=wjm5k6Kf-RU

https://www.youtube.com/watch?v=YQtDiX2Dbr0

#Flow of oil and gas in wells
Simulating flow in oil and gas wells is a complex task that integrates principles from fluid dynamics, thermodynamics, rock mechanics, and more. The specific equations used can vary based on the details of the simulation and the assumptions made. However, here's a basic summary of the primary equations involved:

1. **Darcy's Law for Single Phase Flow**:
   Describes the flow of a fluid through a porous medium.
   $ q = -kA\frac{\Delta P}{\mu L} $
   Where:
   - $ q $ = flow rate (m³/s)
   - $ k $ = permeability of the rock (m²)
   - $ A $ = cross-sectional area perpendicular to the flow (m²)
   - $ \Delta P $ = pressure drop across the length $ L $ (Pa)
   - $ \mu $ = fluid viscosity (Pa.s)
   - $ L $ = length over which the pressure drop occurs (m)

2. **Multiphase Flow**:
   The flow of oil, gas, and sometimes water concurrently in a well can be described using relative permeabilities and phase saturations. The generalized Darcy's law for a phase $ i $ becomes:
   $ q_i = -\frac{k k_{ri}}{\mu_i}A\frac{\Delta P}{L} $
   Where:
   - $ k_{ri} $ = relative permeability for phase $ i $
   - $ \mu_i $ = viscosity of phase $ i $

3. **Conservation of Mass**:
   For any phase $ i $:
   $ \frac{\partial (\phi \rho_i S_i)}{\partial t} + \nabla \cdot (\rho_i \mathbf{v}_i) = q_i $
   Where:
   - $ \phi $ = porosity of the rock
   - $ \rho_i $ = density of phase $ i $
   - $ S_i $ = saturation of phase $ i $
   - $ \mathbf{v}_i $ = Darcy velocity of phase $ i $
   - $ q_i $ = source/sink term (e.g., well injection/production rate)

4. **Conservation of Momentum**:
   The most basic form is the mechanical energy balance or the hydraulic gradient equation:
   $ \frac{\Delta P}{L} = \rho g h + \frac{1}{2} \rho v^2 + f \frac{\rho v^2}{2D} $
   Where:
   - $ \rho $ = fluid density
   - $ g $ = gravitational acceleration
   - $ h $ = elevation head (height difference)
   - $ v $ = fluid velocity
   - $ D $ = pipe diameter
   - $ f $ = friction factor

5. **PVT Relations**:
   Equations that relate pressure, volume, and temperature for oil and gas. These can be empirical correlations, cubic equations of state (like Peng-Robinson), or tabulated lab data.

6. **Wellbore Flow Performance (Inflow and Outflow Curves)**:
   $ P_{wf} = P_r - q \times IPR $ for inflow
   $ P_{wf} = P_{wh} + \Delta P_{friction} + \Delta P_{elevation} $ for outflow
   Where:
   - $ P_{wf} $ = flowing bottomhole pressure
   - $ P_r $ = reservoir pressure
   - $ P_{wh} $ = wellhead pressure
   - $ IPR $ = Inflow Performance Relationship (function or curve describing reservoir deliverability)

7. **Temperature Profiles**:
   Considering joule-thomson effect, frictional heating, geothermal gradient, etc., can be essential for deep wells or when simulating well interventions.

Simulation of flow in wells often involves coupling these equations and solving them iteratively. Advanced simulations might incorporate more complex phenomena like phase transitions, non-Newtonian fluid behavior, thermal effects, and rock-fluid interactions. Commercial reservoir and well simulators often solve these equations using numerical techniques like finite difference or finite element methods.


#Beggs and Brills method


The Beggs and Brill method is a widely recognized empirical method used for predicting multiphase flow in vertical and inclined wellbore tubing, primarily in the oil and gas industry. Developed by J.R. Brill and H. Beggs at the University of Tulsa in the 1960s, this method helps in predicting pressure drops for flowing gas-liquid mixtures in pipes.

Here's a summary of the key equations and concepts associated with the Beggs and Brill method:

1. **Flow Pattern Prediction**:
   Flow pattern is an essential component for subsequent calculations. The method categorizes flow patterns into:
   - Bubble Flow
   - Slug Flow
   - Transition Flow (only in upward flow in inclined and vertical pipes)
   - Annular Flow
   A set of dimensionless groups and curves derived from experimental data are used to determine which flow regime is present.

2. **Liquid Holdup, $ \epsilon $**:
   The liquid holdup represents the fraction of the pipe cross-sectional area that's occupied by the liquid phase. An empirical correlation derived from experimental data provides the liquid holdup:
   $ \epsilon = f(Fr, N_{LV}, \theta, \rho_L, \rho_G) $
   Where:
   - $ Fr $ is the Froude number
   - $ N_{LV} $ is a dimensionless liquid velocity number
   - $ \theta $ is the angle of inclination of the pipe
   - $ \rho_L $ and $ \rho_G $ are the densities of the liquid and gas phases, respectively.

3. **Frictional Pressure Drop**:
   The frictional pressure drop, $ \Delta P_f $, is given as:
   $ \Delta P_f = f_p \times \frac{L}{D} \times \frac{\rho_m v^2}{2} $
   Where:
   - $ f_p $ is the two-phase friction factor, which is determined using empirical correlations based on flow regime.
   - $ L $ is the length of the pipe segment.
   - $ D $ is the pipe diameter.
   - $ \rho_m $ is the two-phase mixture density.
   - $ v $ is the mixture velocity.

4. **Gravitational Pressure Drop**:
   The gravitational pressure drop, $ \Delta P_g $, due to the hydrostatic head of the fluid column, is calculated as:
   $ \Delta P_g = \epsilon \rho_L g h + (1 - \epsilon) \rho_G g h $
   Where:
   - $ g $ is the acceleration due to gravity.
   - $ h $ is the height of the pipe segment.

5. **Total Pressure Drop**:
   The total pressure drop, $ \Delta P $, across the pipe segment can then be obtained as the sum of the frictional and gravitational pressure drops:
   $ \Delta P = \Delta P_f + \Delta P_g $

The Beggs and Brill method, while empirical, provides a practical way of estimating pressure drops in multiphase flow scenarios, especially when other detailed mechanistic models might be computationally expensive or require more detailed input data. However, users should be aware of its limitations, and it might not be appropriate for all situations, especially in non-standard conditions or when very accurate predictions are necessary.

#NeqSim example using Beggs and Brills method

In [4]:
inletPressure = 30.0
inletTemperature = 45.2
fluid1 = createfluid('light oil')
fluid1.setPressure(inletPressure, "bara")
fluid1.setTemperature(inletTemperature, "C")
TPflash(fluid1)
printFrame(fluid1)

| 0                  | 1           | 2                  | 3                  | 4   | 5   | 6               |
|:-------------------|:------------|:-------------------|:-------------------|:----|:----|:----------------|
|                    | total       | GAS                | OIL                |     |     |                 |
| nitrogen           | 8.60585E-3  | 1.66723E-2         | 1.05336E-3         |     |     | [mole fraction] |
| CO2                | 1.72117E-2  | 2.3679E-2          | 1.11565E-2         |     |     | [mole fraction] |
| methane            | 4.47504E-1  | 7.98178E-1         | 1.19175E-1         |     |     | [mole fraction] |
| ethane             | 9.46644E-2  | 1.18847E-1         | 7.20223E-2         |     |     | [mole fraction] |
| propane            | 4.30293E-2  | 3.17258E-2         | 5.36125E-2         |     |     | [mole fraction] |
| i-butane           | 8.60585E-3  | 3.7573E-3          | 1.31455E-2         |     |     | [mole fraction] |
| n-butane         

In [18]:
from neqsim.process import stream

stream1 = stream(fluid1)
stream1.setFlowRate(0.5, 'MSm3/day')
stream1.setPressure(105.0, 'bara')
stream1.setTemperature(75.0, 'C')

flowline1 = jNeqSim.processSimulation.processEquipment.pipeline.PipeBeggsAndBrills(stream1);
flowline1.setDiameter(0.35);
flowline1.setPipeWallRoughness(15e-6);
flowline1.setLength(1200);
flowline1.setElevation(1200.0);
flowline1.setNumberOfIncrements(5);
flowline1.run()

print('inlet pressure ', flowline1.getInletStream().getPressure(), 'bara')
print('outlet pressure ', flowline1.getOutStream().getPressure(), 'bara')
print('flow regime ', flowline1.getFlowRegime())

inlet pressure  105.0 bara
outlet pressure  43.4185016414717 bara
flow regime  TRANSITION
